In [1]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

init_db(2000000)

execute_query("SHOW TABLES", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Products table")
execute_query("DESC products", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Brands table")
execute_query("DESC availability", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Brands table")
execute_query("DESC brands", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Categories table")
execute_query("DESC categories", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Colors table")
execute_query("DESC colors", database=get_database_name(), print_as_df=True, show_metrics=False)

Created empty database
Created empty tables
Starting to insert data

Starting insert into 'products' (2000000 rows)...
Finished inserting 2000000 rows into 'products' in 34.74s.


,Tables_in_test_database
0,availability
1,brands
2,categories
3,colors
4,products


Products table


,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,,None,
2,description,text,YES,,None,
3,brand_id,int,YES,MUL,None,
4,category_id,int,YES,MUL,None,
5,price,float,YES,,None,
6,currency,varchar(10),YES,,None,
7,stock,int,YES,,None,
8,ean,varchar(50),YES,,None,
9,color_id,int,YES,MUL,None,


Brands table


...

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,UNI,None,


Categories table


,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,UNI,None,


Colors table


,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(100),YES,UNI,None,


In this notebook i want to playaround with using multiple keys in a index. 
I hope to find that i could have a query with different columns used in a where clause and i could have a single index on of these and it should be faster than having none, but having two keys should improve the performance

In [23]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
SELECT * FROM products WHERE size = 'Large' AND price < 50;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 5617 rows fetched, 729.34 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ALL,None,None,None,None,1977361,3.33,Using where


In [24]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_size ON products(size);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE size = 'Large' AND price < 50;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 5617 rows fetched, 303.01 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,idx_size,idx_size,403,const,223618,33.33,Using where


In [25]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_price ON products(price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE size = 'Large' AND price < 50;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 5617 rows fetched, 690.22 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,range,idx_price,idx_price,5,None,202564,10.0,Using index condition; Using where; Using MRR


In [43]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_size_and_price ON products(size, price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE size = 'Large' AND price < 50;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 5617 rows fetched, 128.93 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,range,idx_size_and_price,idx_size_and_price,408,None,5617,100.0,Using index condition


In [22]:
execute_query("select count(*) from products where price < 50", database=get_database_name(), print_as_df=True, show_metrics=False)
execute_query("select count(*) from products where size = 'Large'", database=get_database_name(), print_as_df=True, show_metrics=False)

,count(*)
0,98019


,count(*)
0,111336


So this seems to show that using a single index can speed thing up (depending on the index and how you query) but two would still be better.
And just for fun i want to add another key to the index to see if this would make it slower. My assumption is that it shouldn't affect the query time

In [44]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_size_and_price_and_name ON products(size, price, name);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE size = 'Large' AND price < 50;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 5617 rows fetched, 181.01 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,range,idx_size_and_price_and_name,idx_size_and_price_and_name,408,None,10734,100.0,Using index condition


It does actually seem to affect it quite a bit. And i dont understand why.

What looks weird is that the rows from the explain is twice s big eventhough the query is the same. 
I dont know if the extra column in the index will mess up the execution planners calulation.